Carico le librerie che mi servono, carico il dataset e stampo le dimensioni

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = pd.read_csv(
    r'.\data_recording.csv')
df.shape

In [ ]:
df.dtypes

Il dataset ha 31646 righe e 46 colonne, visualizzo le prime 10

In [ ]:
df.head(10)

Visualizzo anche le ultime 10

In [ ]:
df.tail(10)

Controllo se ci sono valori mancanti

In [ ]:
df.isnull().values.any()

Controllo se Time contiene valori duplicati

In [ ]:
print(len(df['Time'].unique()))
print(df.Time.unique().all())

Tutti i valori di time sono unici, quindi posso usare Time come indice e considerare il dataset come una time series

In [ ]:
df.set_index("Time", inplace=True)
df.head(10)

Visualizzo i grafici di tutte le colone in funzione di Time

In [ ]:
fig = make_subplots(cols=11, rows=4, subplot_titles=list(df.columns))
x = 0
for r in range(4):
    for c in range(11):
        fig.add_trace(go.Scatter(x=df.index, y=df[df.columns[x]], line=dict(
            width=.5), opacity=1, name=df.columns[x]), row=r+1, col=c+1)
        x += 1
fig.update_layout(showlegend=False)
fig.show()


Dai grafici sembra che i valori di alcune colonne siano tutti costanti

In [ ]:
control = [3, 11, 12, 13, 14, 16, 24, 25,
           26, 28, 29, 30, 31, 32, 39, 40, 41, 42]
fig = make_subplots(cols=6, rows=3, subplot_titles=list(df.columns[control]))
x = 0
for r in range(3):
    for c in range(6):
        fig.add_trace(go.Scatter(x=df.index, y=df.iloc[:, control[x]], line=dict(
            width=.9), opacity=1, name=df.columns[x]), row=r+1, col=c+1)
        x += 1
        if x == 18:
            break
fig.update_layout(showlegend=False)
fig.show()


Tutte queste colonne contengono valori 0 eccetto td_mildTurnDetected, quindi possiamo rimuoverle.
td_mildTurnDetected costante a 1 puo significare che la barca durante il test si stava girando leggermente probabilmente perche il test è stato condotto in acqua, detto questo la colonna non verra più usata quindi verra rimossa.

In [ ]:
df = df.drop(labels=df.columns[control],axis=1)
df.head(10)

A quest punto normalizzo i dati in modo che varino da 0 a 1

In [ ]:
df = (df-df.min())/(df.max()-df.min())
df.head(10)

In [ ]:
fig = make_subplots(cols=7, rows=4, subplot_titles=list(df.columns))
x = 0
for r in range(4):
    for c in range(7):
        fig.add_trace(go.Scatter(x=df.index, y=df[df.columns[x]], line=dict(
            width=.6), opacity=0.8, name=df.columns[x]), row=r+1, col=c+1)        
        x += 1
        if x == 27:
            break
fig.update_layout(showlegend=False)
fig.show()


In [ ]:
fig = go.Figure(data=go.Heatmap(z=df.corr(), x=df.columns, y=df.columns,
                colorscale='RdBu', text=df.corr(), texttemplate="%{text:.1f}", textfont={"size": 7}))
fig.update_layout(font=dict(size=7))
fig.show()

Dalla descrizione del dataset fornita so che la variabile functions indica se lo stabilizzatore era acceso (1) o spento (0) quindi creo dei grafici per vedere se lo stabilizzatore influisce sui dati.
Come dati da paragonare scelgo alcuni valori che ad occhio sembrano variare al variare di functions.

In [ ]:
control = [2, 6, 7, 8, 9, 19, 20, 21, 22, 15, 18]
fig = fig = make_subplots(
    cols=4, rows=3, subplot_titles=list(df.columns[control]))
x = 0
for r in range(3):
    for c in range(4):
        fig.add_trace(go.Scatter(x=df.index, y=df['functions'], opacity=1, line=dict(
            width=.8, color='black'), name='functions'), row=r+1, col=c+1)
        fig.add_trace(go.Scatter(x=df.index, y=df.iloc[:, control[x]], line=dict(
            width=.5), opacity=1, name=df.columns[control[x]]), row=r+1, col=c+1)
        x += 1
        if x == 11:
            break
fig.update_layout(showlegend=False)
fig.show()


Dat grafici sembra che:  
-ls_listAng, gyro_X e ls_rollRate si riducano quando lo stabilizzatore è acceso.  
-servo[0].posDem, servo[0].pos, servo[1].posDem, servo[1].pos, a_portRollFin, a_PortRollFinIn a_stbdRollFin, a_StbdRollFinIn e gyro_X aumentino quando lo stabilizzatore e acceso

Analizziamo altri dati:

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['se_speed'],  opacity=1,  line=dict(
    width=.7, color='blue'),marker=dict(size=.7),mode='lines+markers', name='se_speed'))
fig.add_trace(go.Scatter(x=df.index, y=df['GPS_speed'], opacity=.8, line=dict(
    width=.7, color='red'), name='GPS_speed'))
fig.show()


La velocita misurata dalla barca e quella misurata dal GPS sono simili, con l'unica differenza che GPS_speed è presa a campione ogni tot millisecondi

In [ ]:
control = [4,5,11]
col = ['red','blue','yellow']
titles = list(df.columns[control])
titles.append('ls_trueTurnAng e al_trueTurnTarget')
fig = fig = make_subplots(
    cols=2, rows=2, subplot_titles=titles)
x = 0
for r in range(2):
    for c in range(2):
        fig.add_trace(go.Scatter(x=df.index, y=df['se_speed'], opacity=.8, line=dict(
            width=.8, color='green'), name='se_speed'), row=r+1, col=c+1)
        fig.add_trace(go.Scatter(x=df.index, y=df.iloc[:, control[x]], line=dict(
            width=.5,color=col[x]), opacity=1, name=df.columns[control[x]]), row=r+1, col=c+1)
        x += 1
        if x == 3:
            break
fig.add_trace(go.Scatter(x=df.index, y=df['al_trueTurnTarget'], opacity=1, line=dict(
            width=.8, color='red'), name='al_trueTurnTarget'), row=2, col=2)
fig.add_trace(go.Scatter(x=df.index, y=df['ls_trueTurnAng'], opacity=1, line=dict(
            width=.8, color='blue'), name='ls_trueTurnAng'), row=2, col=2)
fig.update_layout(showlegend=False)
fig.show()

ls_trueTurnAng e al_trueTurnTarget sono molto simili, lo stesso vale per td_turnDetectedSourceVal e se_speed

In [ ]:
fig = fig = make_subplots(cols=3, rows=1,subplot_titles=['accel_X e gyro_X','accel_Y e gyro_Y','accel_Z e gyro_Z'])
fig.add_trace(go.Scatter(x=df.index, y=df['accel_X'], opacity=.7, line=dict(
    width=.5, color='red'),  mode='lines'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['gyro_X'],  opacity=.7, line=dict(
    width=.5, color='blue'), mode='lines'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['accel_Y'], opacity=.7, line=dict(
    width=.5, color='red'),  mode='lines'), row=1, col=2)
fig.add_trace(go.Scatter(x=df.index, y=df['gyro_Y'],  opacity=.7, line=dict(
    width=.5, color='blue'), mode='lines'), row=1, col=2)
fig.add_trace(go.Scatter(x=df.index, y=df['accel_Z'], opacity=.7, line=dict(
    width=.5, color='red'),  mode='lines'), row=1, col=3)
fig.add_trace(go.Scatter(x=df.index, y=df['gyro_Z'],  opacity=.7, line=dict(
    width=.5, color='blue'), mode='lines'), row=1, col=3)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['ls_yawAng'], opacity=.7, line=dict(
    width=.5, color='red'),  mode='lines', name='ls_yawAng'))
fig.add_trace(go.Scatter(x=df.index, y=df['ls_yawRate'],  opacity=.7, line=dict(
    width=.5, color='blue'), mode='lines', name='ls_yawRate'))
fig.show()


In [ ]:
fig = go.Figure([go.Scatter(x=df.index, y=df['GPS_cog'], opacity=.7, line=dict(
    width=1, color='red'),  mode='lines', name='GPS_cog')])
fig.add_trace(go.Scatter(x=df.index, y=df['al_trueTurnTarget'], opacity=1, line=dict(
            width=.8, color='blue'),name='al_trueTurnTarget'))
fig.show()

la barca parte posizionata verso sud-ovest, si gira a sud poi verso nord e alla fine rimane verso sud-est